In [53]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Fetch the data
data = yf.download('AAPL', start='2022-06-01', end='2023-01-01')
close_prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)

# Create sequences
def create_dataset(dataset, look_back=60):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

X, y = create_dataset(close_prices_scaled)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


[*********************100%%**********************]  1 of 1 completed


In [54]:
from deap import base, creator, tools, algorithms
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import random

# Define the fitness function
def evalLSTM(individual):
    n_layers, n_units, dropout_rate, learning_rate, batch_size = individual
    model = Sequential()
    model.add(LSTM(units=n_units, return_sequences=True if n_layers > 1 else False, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(dropout_rate))
    for _ in range(1, n_layers):
        model.add(LSTM(units=n_units, return_sequences=True if _ < n_layers - 1 else False))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=int(batch_size), verbose=1)  # Reduced epochs for faster execution
    predictions = model.predict(X_val)
    mse = np.mean(np.square(y_val - predictions))
    return (mse,)

# Configure the GA
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("attr_layers", random.randint, 1, 3)
toolbox.register("attr_units", random.randint, 50, 150)
toolbox.register("attr_dropout", random.uniform, 0.1, 0.5)
toolbox.register("attr_lr", random.uniform, 0.0001, 0.01)
toolbox.register("attr_batch_size", random.choice, [32, 64, 128])
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_layers, toolbox.attr_units, toolbox.attr_dropout, toolbox.attr_lr, toolbox.attr_batch_size), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evalLSTM)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=[1, 50, 0.1, 0.0001, 32], up=[3, 150, 0.5, 0.01, 128], indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)


c:\Users\Snehal\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Snehal\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [55]:
population = toolbox.population(n=10)  # Smaller population for quicker execution
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats, halloffame=hof, verbose=True)

best_params = hof[0]
print("Best parameters found are:", best_params)


Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 0.2062 - accuracy: 0.0145
Epoch 2/5
1/1 [==============================] - 1s 545ms/step - loss: 0.1148 - accuracy: 0.0145
Epoch 3/5
1/1 [==============================] - 1s 554ms/step - loss: 0.4227 - accuracy: 0.0145
Epoch 4/5
1/1 [==============================] - 1s 604ms/step - loss: 0.1338 - accuracy: 0.0145
Epoch 5/5
1/1 [==============================] - 1s 988ms/step
Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 0.1881 - accuracy: 0.0145
Epoch 2/5
1/1 [==============================] - 1s 715ms/step - loss: 0.7131 - accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 1s 708ms/step - loss: 0.3503 - accuracy: 0.0145
Epoch 4/5
1/1 [==============================] - 1s 704ms/step - loss: 0.1428 - accuracy: 0.0145
Epoch 5/5
1/1 [==============================] - 1s 1s/step
Epoch 1/5
2/2 [==============================] - 2s 27ms/step - loss: 0.1128 - accuracy: 0.0145
Epoch 2

ValueError: non-integer arg 1 for randrange()

In [56]:
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from deap import base, creator, tools, algorithms
import random

# Fetch stock data from Yahoo Finance
data = yf.download('AAPL', start='2022-06-01', end='2023-01-01')
close_prices = data['Close'].values.reshape(-1, 1)

# Data preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)

# Helper function to create sequences
def create_dataset(dataset, look_back=60):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

X, y = create_dataset(close_prices_scaled)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Splitting dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Genetic Algorithm Setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_layers", random.randint, 1, 3)
toolbox.register("attr_units", random.randint, 50, 150)
toolbox.register("attr_dropout", lambda: random.uniform(0.1, 0.5))
toolbox.register("attr_lr", lambda: random.uniform(0.0001, 0.01))
toolbox.register("attr_batch_size", lambda: random.choice([32, 64, 128]))
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_layers, toolbox.attr_units, toolbox.attr_dropout, toolbox.attr_lr, toolbox.attr_batch_size), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evalLSTM)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=[1, 50, 0, 0, 32], up=[3, 150, 0, 0, 128], indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Evaluation function for LSTM
def evalLSTM(individual):
    n_layers, n_units, dropout_rate, learning_rate, batch_size = individual
    model = Sequential()
    model.add(LSTM(units=int(n_units), return_sequences=(n_layers > 1), input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(dropout_rate))
    for _ in range(1, n_layers):
        model.add(LSTM(units=int(n_units), return_sequences=(_ < n_layers - 1)))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=5, batch_size=int(batch_size), verbose=0)
    predictions = model.predict(X_val)
    mse = np.mean(np.square(y_val - predictions))
    return (mse,)

# Run GA Optimization
def main():
    population = toolbox.population(n=10)
    hof = tools.HallOfFame(1)

    for gen in range(10):  # Number of generations
        population = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
        fits = toolbox.map(toolbox.evaluate, population)
        for fit, ind in zip(fits, population):
            ind.fitness.values = fit
        
        population = toolbox.select(population, len(population))
        top_individual = tools.selBest(population, 1)[0]
        print(f"Best individual in generation {gen}: {top_individual}, Fitness (MSE): {top_individual.fitness.values[0]}")
        
        hof.update(population)

    return population, hof


final_pop, hall_of_fame = main()
print("Best found individual:", hall_of_fame[0])
print("With fitness (MSE):", hall_of_fame[0].fitness.values[0])


[*********************100%%**********************]  1 of 1 completed
c:\Users\Snehal\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Snehal\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 0.1915 - accuracy: 0.0145
Epoch 2/5
1/1 [==============================] - 0s 64ms/step - loss: 0.0874 - accuracy: 0.0145
Epoch 3/5
1/1 [==============================] - 0s 66ms/step - loss: 0.0294 - accuracy: 0.0145
Epoch 4/5
1/1 [==============================] - 0s 64ms/step - loss: 0.0282 - accuracy: 0.0145
Epoch 5/5
1/1 [==============================] - 1s 542ms/step
Epoch 1/5
2/2 [==============================] - 2s 45ms/step - loss: 0.2956 - accuracy: 0.0145
Epoch 2/5
2/2 [==============================] - 0s 43ms/step - loss: 0.0875 - accuracy: 0.0145
Epoch 3/5
2/2 [==============================] - 0s 43ms/step - loss: 0.1527 - accuracy: 0.0145
Epoch 4/5
2/2 [==============================] - 0s 56ms/step - loss: 0.0996 - accuracy: 0.0145
Epoch 5/5
1/1 [==============================] - 1s 588ms/step
Epoch 1/5
2/2 [==============================] - 7s 121ms/step - loss: 0.1771 - accuracy: 0.0145
Epoch 2/5
2